In [4]:
from vnstock import Company
from vnstock import Listing
from vnstock import Finance
company = Company(symbol = "SVC", source="TCBS")
company2 = Company(symbol="TRC", source="VCI")
listing = Listing()




In [6]:
company2.news()

,id,news_title,news_sub_title,friendly_sub_title,news_image_url,news_source_link,created_at,public_date,updated_at,lang_code,news_id,news_short_content,news_full_content,close_price,ref_price,floor,ceiling,price_change_pct
0,8469694,TRC: Thông báo thay đổi số lượng cổ phiếu có q...,,,https://cdn.fiingroup.vn/medialib/82/I/2024/12...,https://www.hsx.vn/vi/tin-tuc/trc-thong-bao-th...,None,1758204511000,None,vi,11714845,,<p>Công ty Cổ phần Cao su Tây Ninh thông báo t...,71000.0,71500.0,66500.0,76500.0,-0.006993
1,8238475,TRC: Thông báo giao dịch bán cổ phiếu quỹ,,,https://cdn.fiingroup.vn/medialib/82/I/2024/12...,https://www.hsx.vn/vi/tin-tuc/trc-thong-bao-gi...,None,1755264143000,None,vi,11674612,,<p>Công ty Cổ phần Cao su Tây Ninh thông báo g...,66300.0,67200.0,62500.0,71900.0,-0.013393
2,8126263,TRC: Giải trình chênh lệch LNST 6 tháng đầu nă...,,,https://cdn.fiingroup.vn/medialib/82/I/2024/12...,https://www.hsx.vn/vi/tin-tuc/trc-giai-trinh-c...,None,1754077920000,None,vi,11660045,,<p>Công ty Cổ phần Cao su Tây Ninh giải trình ...,NaN,NaN,NaN,NaN,NaN
3,8093107,TRC: Nghị quyết HĐQT về kết quả thực hiện các ...,,,https://cdn.fiingroup.vn/medialib/82/I/2024/12...,https://www.hsx.vn/vi/tin-tuc/trc-nghi-quyet-h...,None,1754047579000,None,vi,11659419,,<p>Công ty Cổ phần Cao su Tây Ninh công bố Ngh...,64500.0,65900.0,61300.0,70500.0,-0.021244
4,8079783,TRC: Báo cáo tình hình quản trị 6 tháng đầu nă...,,,https://cdn.fiingroup.vn/medialib/82/I/2024/12...,https://www.hsx.vn/vi/tin-tuc/trc-bao-cao-tinh...,None,1753899844000,None,vi,11656111,,<p>Công ty Cổ phần Cao su Tây Ninh Báo cáo tìn...,66300.0,65700.0,61200.0,70200.0,0.009178
5,7970079,"Cao su Tây Ninh báo lãi quý II tăng 172%, hoàn...",,,,https://baodautu.vn/cao-su-tay-ninh-bao-lai-qu...,None,1752835920000,None,vi,11638632,CTCP Cao su Tây Ninh (HOSE: TRC) vừa công bố k...,"<p>Theo báo cáo tài chính vừa công bố, trong q...",70900.0,68800.0,64000.0,73600.0,0.030523
6,7962760,TRC: Thông báo thay đổi môc dấu công ty,,,https://cdn.fiingroup.vn/medialib/82/I/2024/12...,https://www.hsx.vn/vi/tin-tuc/trc-thong-bao-th...,None,1752774234000,None,vi,11637370,,<p>Công ty Cổ phần Cao su Tây Ninh thông báo t...,68800.0,68200.0,63500.0,72900.0,0.008798
7,7955328,"Cao su Tây Ninh (TRC) lãi gấp 3,7 lần trong nử...",,,https://cdn.fiingroup.vn/medialib/127379/I/202...,https://www.tinnhanhchungkhoan.vn/cao-su-tay-n...,None,1752682740000,None,vi,11635631,"&nbsp;Luỹ kế nửa đầu năm 2025, Cao su Tây Ninh...","<div class=""banner clearfix"" data-platform=""1""...",68200.0,67100.0,62500.0,71700.0,0.016393
8,7955141,TRC: Giải trình biến động LNST quý 2/2025 so v...,,,https://cdn.fiingroup.vn/medialib/82/I/2024/12...,https://www.hsx.vn/vi/tin-tuc/trc-giai-trinh-b...,None,1752601200000,None,vi,11634179,,<p>Công ty Cổ phần Cao su Tây Ninh thông báo g...,67100.0,67500.0,62800.0,72200.0,-0.005926
9,7955171,TRC: Thông báo ký kết hợp đồng kiểm toán BCTC ...,,,https://cdn.fiingroup.vn/medialib/82/I/2024/12...,https://www.hsx.vn/vi/tin-tuc/trc-thong-bao-ky...,None,1752600634000,None,vi,11634222,,<p>Công ty Cổ phần Cao su Tây Ninh thông báo ...,67100.0,67500.0,62800.0,72200.0,-0.005926
